In [1]:
import numpy as np
import pandas as pd
import os
import csv
from datetime import timedelta, datetime, date
import datetime as dt
import pickle

In [2]:
#Import ESC File#
esc_file = pd.read_csv("ESC 08072020.csv",dtype={'Pernr SAP': str})
esc_file = esc_file[['Pernr SAP', 'Covid Y/N','Work Location','Ticket Created - Date']]

#Import roster file
roster_file = pd.read_csv("Roster_08072020_workbook.csv",dtype={'Employee PERNR - Key': str,'WL County Fips':str, "HL County Fips":str })
# roster_file = roster_file[['Employee PERNR','Employee PERNR - Key','Annual ABBR',
#                           'Annual Salary','Date of Birth (*)', 'Division (As of Today)','Division-1 (As of Today)','Division-2 (As of Today)',
#                           'Employee Sub Group','Employment Status','Headcount','Home Location City','Home Location State','Home Location ZIP (*)',
#                           'Job Code','Job Family','Job Function','Job Group 2','Job Group 3','Job Level','Org Key (As of Today)','Org Unit','Original Hire Date','Virtual Employee Type',
#                           'Work Location City','Work Location State','Work Location Zip'
#                           ,'WL Zip Truncated','WL County Fips','WL County Name','HL Zip Truncated','HL County Fips','HL County Name']]

roster_file = roster_file[['Employee PERNR - Key','WL Zip Truncated','WL County Fips','WL County Name','HL Zip Truncated',
                           'HL County Fips','HL County Name','Headcount','Job Function','Job Group 1']]
#Import base file
base = pd.read_csv("Prevalence_base.csv", dtype={'fips':str,'Population':int})
# print(esc_file.head())
# print(roster_file.head())
# print(base.head())

C:\Users\dherlu494\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (25,29,76) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Change name of Pernr columns in both files

roster_file1 = roster_file.rename({'Employee PERNR - Key':'PERNR'}, axis = 1)
esc_file1 = esc_file.rename({'Pernr SAP':'PERNR','Ticket Created - Date':'Date'},axis = 1)

In [4]:
print(esc_file1.head())
# print(roster_file1.head())

      PERNR  Covid Y/N Work Location       Date
0  10001471          0         PAL78   5/1/2020
1  10001709          0         NJA01   5/1/2020
2  10001712          0         NJA01  5/13/2020
3  10001715          0         NJM08  5/21/2020
4  10001728          0         NJA01   5/1/2020


In [5]:
merge = roster_file1.merge(esc_file1, 
                          how = 'left', on = 'PERNR')

print(merge.head())

      PERNR  WL Zip Truncated WL County Fips       WL County Name  \
0  10000054             20814          24031    Montgomery County   
1  10000063             19103          42101  Philadelphia County   
2  10000401             19053          42017         Bucks County   
3  10000480             80112          08005      Arapahoe County   
4  10000720             80112          08005      Arapahoe County   

  HL Zip Truncated HL County Fips     HL County Name  Headcount  \
0            22033          51059     Fairfax County          1   
1             8053          34005  Burlington County          1   
2            19118          42091  Montgomery County          1   
3            80126          08035     Douglas County          1   
4            80209          08031      Denver County          1   

               Job Function Job Group 1  Covid Y/N Work Location Date  
0         Advertising Sales    Ad Sales        NaN           NaN  NaN  
1      Finance & Accounting   All Othe

In [7]:
#aggregate at fips level#
WL = merge[['WL County Fips','Covid Y/N','Headcount']]
WL_df = WL.groupby(['WL County Fips'], as_index=False).sum()
WL_df = WL_df.rename({'Covid Y/N':'WL_Covid','Headcount':'WL_Count','WL County Fips': 'fips'}, axis = 1)

HL = merge[['HL County Fips','Covid Y/N','Headcount']]
HL_df = HL.groupby(['HL County Fips'], as_index=False).sum()
HL_df = HL_df.rename({'Covid Y/N':'HL_Covid','Headcount':'HL_Count','HL County Fips': 'fips'}, axis = 1)

# print(WL_df.head())
# print(HL_df.head())


In [8]:
# #Grouping by Fips Covid numbers by Job Function
# ##FIELD OPERATIONS
# WL_FO = merge[merge["Job Function"]== "Field Operations"]
# WL_FOs = WL_FO[['WL County Fips','Covid Y/N']]
# WL_df_FO = WL_FOs.groupby(['WL County Fips'], as_index=False).sum()
# WL_df_FOs = WL_df_FO.rename({'Covid Y/N':'WL_FO_Covid','WL County Fips': 'fips'}, axis = 1)
# print(WL_df_FOs.head())

# ##CUSTOMER SERVICE##
# WL_CS = merge[merge["Job Function"]== "Customer Service"]
# WL_CSs = WL_CS[['WL County Fips','Covid Y/N']]
# WL_df_CS = WL_CSs.groupby(['WL County Fips'], as_index=False).sum()
# WL_df_CSs = WL_df_CS.rename({'Covid Y/N':'WL_CS_Covid','WL County Fips': 'fips'}, axis = 1)
# print(WL_df_CSs.head())


# ##PRODUCT SALES##
# WL_PS = merge[merge["Job Function"]== "Product Sales"]
# WL_PSs = WL_PS[['WL County Fips','Covid Y/N']]
# WL_df_PS = WL_PSs.groupby(['WL County Fips'], as_index=False).sum()
# WL_df_PSs = WL_df_PS.rename({'Covid Y/N':'WL_PS_Covid','WL County Fips': 'fips'}, axis = 1)
# print(WL_df_PSs.head())


# ##ENGINEERING##
# WL_ENG = merge[merge["Job Function"]== "Engineering"]
# WL_ENGs = WL_ENG[['WL County Fips','Covid Y/N']]
# WL_df_ENG = WL_ENGs.groupby(['WL County Fips'], as_index=False).sum()
# WL_df_ENGs = WL_df_ENG.rename({'Covid Y/N':'WL_ENG_Covid','WL County Fips': 'fips'}, axis = 1)
# print(WL_df_ENGs.head())


# ##FINANCE & ACCOUNTING
# WL_FA = merge[merge["Job Function"]== "Finance & Accounting"]
# WL_FAs = WL_FA[['WL County Fips','Covid Y/N']]
# WL_df_FA = WL_FAs.groupby(['WL County Fips'], as_index=False).sum()
# WL_df_FAs = WL_df_FA.rename({'Covid Y/N':'WL_FA_Covid','WL County Fips': 'fips'}, axis = 1)
# print(WL_df_FAs.head())

In [9]:
#JOB GROUP 1 #
WL_Tech = merge[merge["Job Group 1"]== "Techs"]
WL_Techs = WL_Tech[['WL County Fips','Covid Y/N','Headcount']]
WL_df_Tech = WL_Techs.groupby(['WL County Fips'], as_index=False).sum()
WL_df_Techs = WL_df_Tech.rename({'Covid Y/N':'WL_Techs_Covid','WL County Fips': 'fips','Headcount':'Tech_Count'}, axis = 1)
print(WL_df_Techs.head())


WL_CAE = merge[merge["Job Group 1"]== "CAE"]
WL_CAES = WL_CAE[['WL County Fips','Covid Y/N','Headcount']]
WL_df_CAE = WL_CAES.groupby(['WL County Fips'], as_index=False).sum()
WL_df_CAES = WL_df_CAE.rename({'Covid Y/N':'WL_CAE_Covid','WL County Fips': 'fips','Headcount':'CAE_Count'}, axis = 1)
print(WL_df_CAES.head())


WL_CableStore = merge[merge["Job Group 1"]== "Cable Stores"]
WL_CableStores = WL_CableStore[['WL County Fips','Covid Y/N','Headcount']]
WL_df_CableS = WL_CableStores.groupby(['WL County Fips'], as_index=False).sum()
WL_df_CableStores = WL_df_CableS.rename({'Covid Y/N':'WL_CABLEStores_Covid','WL County Fips': 'fips','Headcount':'Cable_Count'}, axis = 1)
print(WL_df_CableStores.head())

# WL_BusServ = merge[merge["Job Group 1"]== "Business Services"]
# WL_BusServs = WL_BusServ[['WL County Fips','Covid Y/N']]
# WL_df_BusServ = WL_BusServs.groupby(['WL County Fips'], as_index=False).sum()
# WL_df_BusServs = WL_df_BusServ.rename({'Covid Y/N':'WL_BusServ_Covid','WL County Fips': 'fips'}, axis = 1)
# print(WL_df_BusServs.head())

#JOB GROUP 1 - july only#
WL_Tech1 = thismonth[thismonth["Job Group 1"]== "Techs"]
WL_Techs1 = WL_Tech1[['WL County Fips','Covid Y/N']]
WL_df_Tech1 = WL_Techs1.groupby(['WL County Fips'], as_index=False).sum()
WL_df_Techs1 = WL_df_Tech1.rename({'Covid Y/N':'WL_Techs_july','WL County Fips': 'fips'}, axis = 1)
print(WL_df_Techs1.head())


WL_CAE1 = thismonth[thismonth["Job Group 1"]== "CAE"]
WL_CAES1 = WL_CAE1[['WL County Fips','Covid Y/N']]
WL_df_CAE1 = WL_CAES1.groupby(['WL County Fips'], as_index=False).sum()
WL_df_CAES1 = WL_df_CAE1.rename({'Covid Y/N':'WL_CAE_july','WL County Fips': 'fips'}, axis = 1)
print(WL_df_CAES1.head())


WL_CableStore1 = thismonth[thismonth["Job Group 1"]== "Cable Stores"]
WL_CableStores1 = WL_CableStore1[['WL County Fips','Covid Y/N']]
WL_df_CableS1 = WL_CableStores1.groupby(['WL County Fips'], as_index=False).sum()
WL_df_CableStores1 = WL_df_CableS1.rename({'Covid Y/N':'WL_CABLEStores_july','WL County Fips': 'fips'}, axis = 1)
print(WL_df_CableStores1.head())


    fips  WL_Techs_Covid
0  01019             0.0
1  01061             0.0
2  01077             0.0
3  01089             0.0
4  01097             0.0
    fips  WL_CAE_Covid
0  01089           1.0
1  01097           0.0
2  01125           1.0
3  04019           2.0
4  05035           1.0
    fips  WL_CABLEStores_Covid
0  01019                   0.0
1  01061                   0.0
2  01089                   0.0
3  01097                   0.0
4  01125                   0.0
    fips  WL_Techs_july
0  04019            3.0
1  06001            0.0
2  06007            1.0
3  06013            5.0
4  06019            1.0
    fips  WL_CAE_july
0  01089          1.0
1  06001          1.0
2  06075          0.0
3  06081          0.0
4  08005          2.0
    fips  WL_CABLEStores_july
0  04019                  0.0
1  06007                  0.0
2  06019                  2.0
3  06047                  0.0
4  06053                  0.0


In [10]:
# #Add Code here for joining to FIPS base file to map back if needed#

# df_CCC_cases1 = pd.merge(base, WL_df[['fips','WL_Covid']], on = 'fips', how = 'left')

# df_CCC_cases2 = pd.merge(df_CCC_cases1, HL_df[['fips','HL_Covid']], on = 'fips', how = 'left')

# df_CCC_cases3 = pd.merge(df_CCC_cases2, WL_df_FOs[['fips','WL_FO_Covid']], on = 'fips', how = 'left')

# df_CCC_cases4 = pd.merge(df_CCC_cases3, WL_df_CSs[['fips','WL_CS_Covid']], on = 'fips', how = 'left')

# df_CCC_cases5 = pd.merge(df_CCC_cases4, WL_df_PSs[['fips','WL_PS_Covid']], on = 'fips', how = 'left')

# df_CCC_cases6 = pd.merge(df_CCC_cases5, WL_df_ENGs[['fips','WL_ENG_Covid']], on = 'fips', how = 'left')

# df_CCC_cases7 = pd.merge(df_CCC_cases6, WL_df_FAs[['fips','WL_FA_Covid']], on = 'fips', how = 'left')

# df_CCC_cases8 = pd.merge(df_CCC_cases7, WL_df_Techs[['fips','WL_Techs_Covid']], on = 'fips', how = 'left')

# df_CCC_cases9 = pd.merge(df_CCC_cases8, WL_df_CAES[['fips','WL_CAE_Covid']], on = 'fips', how = 'left')

# df_CCC_cases10 = pd.merge(df_CCC_cases9, WL_df_CableStores[['fips','WL_CABLEStores_Covid']], on = 'fips', how = 'left')

# df_CCC_cases = pd.merge(df_CCC_cases10, WL_df_BusServs[['fips','WL_BusServ_Covid']], on = 'fips', how = 'left')

# print(df_CCC_cases.head(5))

NameError: name 'df_CCC_cases' is not defined

In [11]:
#filtered for only what we care for and july
#Add Code here for joining to FIPS base file to map back if needed#

df_CCC_cases1 = pd.merge(base, WL_df[['fips','WL_Covid''WL_Count']], on = 'fips', how = 'left')

df_CCC_cases2 = pd.merge(df_CCC_cases1, HL_df[['fips','HL_Covid','HL_Count']], on = 'fips', how = 'left')

df_CCC_cases3 = pd.merge(df_CCC_cases2, WL_df_CAES[['fips','WL_CAE_Covid','CAE_Count']], on = 'fips', how = 'left')

df_CCC_cases4 = pd.merge(df_CCC_cases3, WL_df_Techs[['fips','WL_Techs_Covid','Tech_Count']], on = 'fips', how = 'left')

df_CCC_cases = pd.merge(df_CCC_cases4, WL_df_CableStores[['fips','WL_CABLEStores_Covid','Cable_Count']], on = 'fips', how = 'left')


In [12]:
#Change Nulls to 0#
df_CCC_cases1 = df_CCC_cases.fillna(0)
print(df_CCC_cases1.head())

    fips   County    State state_code  Population  Comcast Pop (HL)  \
0  01001  Autauga  Alabama         AL       55869                 0   
1  01003  Baldwin  Alabama         AL      223234                10   
2  01005  Barbour  Alabama         AL       24686                 0   
3  01007     Bibb  Alabama         AL       22394                 4   
4  01009   Blount  Alabama         AL       57826                 2   

   Comcast Pop (WL) CCC < 50 Employees  Field Ops  Customer Service  ...  \
0                 0                Yes          0                 0  ...   
1                 0                Yes          0                 0  ...   
2                 0                Yes          0                 0  ...   
3                 0                Yes          0                 0  ...   
4                 0                Yes          0                 0  ...   

   WL_Covid  HL_Covid  WL_CAE_Covid  WL_CAE_july  WL_Techs_Covid  \
0       0.0       0.0           0.0          0.0

In [13]:
#Pull in NYTimes Data#
#Bring in County Data set
nytimes = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
counties = pd.read_csv(nytimes,dtype={'fips': str})

counties.loc[counties['county'] == 'New York City', 'fips'] = '36061'


In [14]:
#Filter for specific date for prevalence data pull#
counties.sort_values(by=['fips','date'],inplace=True, ascending=True)


counties_filtered = counties[counties['date'].isin(['2020-08-07'])]
print(counties_filtered.head())

              date   county    state   fips  cases  deaths
408508  2020-08-07  Autauga  Alabama  01001   1113      22
408509  2020-08-07  Baldwin  Alabama  01003   3504      25
408510  2020-08-07  Barbour  Alabama  01005    624       5
408511  2020-08-07     Bibb  Alabama  01007    424       4
408512  2020-08-07   Blount  Alabama  01009    909       3


In [15]:
#Merge with the esc_roster dataset

df = pd.merge(df_CCC_cases1, counties_filtered[['fips','cases','deaths']],on = 'fips', how = 'left' )
print(df.head())
# df.to_csv=('mmmk.csv')

    fips   County    State state_code  Population  Comcast Pop (HL)  \
0  01001  Autauga  Alabama         AL       55869                 0   
1  01003  Baldwin  Alabama         AL      223234                10   
2  01005  Barbour  Alabama         AL       24686                 0   
3  01007     Bibb  Alabama         AL       22394                 4   
4  01009   Blount  Alabama         AL       57826                 2   

   Comcast Pop (WL) CCC < 50 Employees  Field Ops  Customer Service  ...  \
0                 0                Yes          0                 0  ...   
1                 0                Yes          0                 0  ...   
2                 0                Yes          0                 0  ...   
3                 0                Yes          0                 0  ...   
4                 0                Yes          0                 0  ...   

   WL_CAE_Covid  WL_CAE_july  WL_Techs_Covid  WL_Techs_july  \
0           0.0          0.0             0.0         

In [ ]:
#Bring in COVID TRACKING data to eventually merge with full dataset
url_all = "https://covidtracking.com/api/v1/states/daily.csv"
testing_all = pd.read_csv(url_all)
testing = testing_all[['date','state','hospitalizedCumulative']]
testing['state_code'] = testing['state']
testing1 = testing[testing['date'].isin(['20200807'])]
print(testing1.head())

In [ ]:
df_testing = pd.merge(df, testing1[['state_code','hospitalizedCumulative']],on = 'state_code', how = 'left')
# print(df_testing.dtypes)

In [ ]:
#Change Nulls to 0#
df_testing1 = df_testing.fillna(0)
# print(df_testing1.dtypes)


In [ ]:
#Prevalence numbers- County
df_testing1['County_Prevalence'] = df_testing1['cases']/df_testing1['Population']
df_testing1['Death/Inf'] = df_testing1['deaths']/df_testing1['cases']
df_testing1['Deaths_Prev'] =df_testing1['deaths']/df_testing1['Population']

#need to divide by state pop#
df_testing1['Hosp/Inf'] = df_testing1['hospitalizedCumulative']/df_testing1['cases']

#Prevalence numbers- Comcast County
df_testing1['WL_Prev'] = df_testing1['WL_Covid']/df_testing1['Comcast Pop (WL)']
df_testing1['HL_Prev'] = df_testing1['HL_Covid']/df_testing1['Comcast Pop (HL)']

#Prevalence by Job Function
# df_testing1['FieldOps_Prev'] = df_testing1['WL_FO_Covid']/df_testing1['Field Ops']
# df_testing1['CusService_Prev'] = df_testing1['WL_CS_Covid']/df_testing1['Customer Service']
# df_testing1['ProductSales_Prev'] = df_testing1['WL_PS_Covid']/df_testing1['Product Sales']
# df_testing1['Engineer_Prev'] = df_testing1['WL_ENG_Covid']/df_testing1['Engineering']
# df_testing1['FinanceAcct_Prev'] = df_testing1['WL_FA_Covid']/df_testing1['Finance & Accounting']

df_testing1['Techs_Prev'] = df_testing1['WL_Techs_Covid']/df_testing1['Techs']
df_testing1['CAE_Prev'] = df_testing1['WL_CAE_Covid']/df_testing1['CAE']
df_testing1['CABLESTORES_Prev'] = df_testing1['WL_CABLEStores_Covid']/df_testing1['Cable Stores']
# df_testing1['BUSSERV_Prev'] = df_testing1['WL_BusServ_Covid']/df_testing1['Business Services']

In [ ]:
print(df_testing3.head())

In [ ]:
#Change Nulls to 0#

df_testing2 = df_testing1.replace([np.inf, -np.inf], np.nan)

df_testing = df_testing2.fillna(0)


In [ ]:
date = '08072020'
# df_final = df_testing3[df_testing3['Comcast Pop (WL)'] >= 50]
df_final = df_testing
df_final.to_csv("Prevalence_"+date+".csv")

In [ ]:
#Create Pickle to Run Linear Regression#
pickle.dump(df_final, open("comcast_regression.sav", "wb"))

# Run code for Previous Month only

In [6]:
#create another df for cases in last month
thismonth = merge[merge['Date'] >= '7/1/2020']

#aggregate at fips level for this month only
WL_july = thismonth[['WL County Fips','Covid Y/N','Headcount']]
WL_df_july1 = WL_july.groupby(['WL County Fips'], as_index=False).sum()
WL_df_july = WL_df_july1.rename({'Covid Y/N':'WL_july','WL County Fips': 'fips'}, axis = 1)

HL_july = thismonth[['HL County Fips','Covid Y/N','Headcount']]
HL_df_july1 = HL_july.groupby(['HL County Fips'], as_index=False).sum()
HL_df_july = HL_df_july1.rename({'Covid Y/N':'HL_july','HL County Fips': 'fips'}, axis = 1)



df_CCC_july1 = pd.merge( ######, WL_df_july[['fips','WL_july']])
df_CCC_july2 = pd.merge( df_CCC_july1, WL_df_july[['fips','WL_july']])

df_CCC_july = df_CCC_july2.fillna(0)

In [ ]:
#New cases in last month--From NYT Data
counties_filtered_lastmonth = counties[counties['date'] >= '2020-08-01']
print(counties_filtered_lastmonth.head())

In [ ]:
#Group by to get new cases 
#take diff of cases
counties_filtered_lastmonth['diffs'] = counties_filtered_lastmonth.cases.diff()
counties_filtered_lastmonth1 = counties_filtered_lastmonth.fillna(0)
print(counties_filtered_lastmonth1.head())

In [ ]:
us_diff_grouped = counties_filtered_lastmonth1.groupby(['fips'], as_index=False).sum()
print(us_diff_grouped.head(25))

In [ ]:
df2 = pd.merge(df_CCC_july, us_diff_grouped[['fips','cases','diffs']])

In [ ]:
#Get Prevalence Info

df2['WL_Prev_july'] = df2['WL_july']/df2['Comcast Pop (WL)']
df2['HL_Prev_july'] = df2['HL_july']/df2['Comcast Pop (HL)']

df2['Techs_july_prev'] = df2['WL_Techs_july']/df2['Techs']
df2['CAE_july_Prev'] = df2['WL_CAE_Covid']/df2['CAE']
df2['Cable_july_Prev'] = df2['WL_CAE_Covid']/df2['CAE']

In [ ]:
df3 = df2.replace([np.inf, -np.inf], np.nan)

df4 = df3.fillna(0)
